<a href="https://colab.research.google.com/github/anujott-codes/Credit-Loan-Approval/blob/main/Loan_Approval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>